In [2]:
import pandas as pd
import re
import mysql.connector as sql
from sqlalchemy import create_engine


def find_date(text):
    """
    text: set of string containing mmm year
    We have created this function to get the month and year from string and convert it to datetime format
    """
    regex = re.compile('(?P<month>[a-zA-Z]+'')\s+(?P<year>\d{4})')
    m = regex.search(text)
   
    dt='01-'+str(m.group('month'))+"-"+str(m.group('year'))    
    return dt



def unstructuredpivottostruduredpivot(excelfilpath):
    """
    excelfilepath: This variable takes excel file path
    In this function we have applied rule based logic to convert unstructured pivot table to structured pivot table
    Note: Few logics are rule based hence if any change in format of excel require fine tuning of code
    """
   
    output_datum_df=pd.DataFrame(columns=['Category','DataItemCode','DataItemName','SubDataItem','State','District','Value'])
   
    df=pd.read_excel("Book1.xlsx",sheet_name="Sheet1")
    data_load_date=find_date(df.columns[0])
    #print(data_load_date)
   
    df=df.iloc[1: , :]
    df.columns=list(df.head(1).ffill(axis=1).values[0])

    df=df.iloc[1: , :]
    df=df.T

    df=df.reset_index()
    df.iloc[0]=df.head(1).ffill(axis=1).values[0]
   
    df=df.T
   
    k=4
   
    model_datum_dct={}
    for i in range(len(df.head(1).values[0])-4):
        j=k+i
        length=len(df.iloc[2::,][j])
        output_datum_df.loc[len(output_datum_df)]=[list(df.iloc[2::,][0].values),list(df.iloc[2::,][1].values),
                                  list(df.iloc[2::,][2].values),list(df.iloc[2::,][3].values),
                                  [df.head(1).values[0][j]]*length,[df.iloc[1:,:].head(1).values[0][j]]*length,
                                  list(df.iloc[2::,][j].values)]
       
       
    output_datum_df=output_datum_df.explode(['Category','DataItemCode','DataItemName','SubDataItem','State','District','Value'])
    output_datum_df['Date']=data_load_date
   
   
   
    return output_datum_df[['Date','Category','DataItemCode','DataItemName','SubDataItem','State','District','Value']]
   
   
   


datum_df=unstructuredpivottostruduredpivot("Book1.xlsx") 



Server='LAPTOP-5N0V7Q2J\SQLEXPRESS'
Database='master'
Driver='ODBC Driver 17 for SQL Server'
Database_Con=f'mssql://@{Server}/{Database}?driver={Driver}'

engine=create_engine(Database_Con)
con=engine.connect()



# datum_df.to_sql(con=my_conn,name='work',if_exists='append',index=False)

datum_df.to_sql('Work',engine,if_exists='append',index=False)

-1

In [6]:
import pandas as pd
import pyodbc

sql_conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                            'SERVER=LAPTOP-5N0V7Q2J\SQLEXPRESS;'
                            'DATABASE=master;'
                            'Trusted_Connection=yes;') 
query = "SELECT * from flight where ID='U2F26'"
df = pd.read_sql(query, sql_conn)

C:\Users\karti\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
df

,ID,Department,Office,Rating
0,U2F26,Finance,New Delhi,3.4
